In [345]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer

### Amount of rows

In [346]:
education = pd.read_csv('education.csv')

education

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,MERCEDES JOSEFINA TORRES GALVEZ,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,MERCEDES JOSEFINA TORRES GALVEZ,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,MERCEDES JOSEFINA TORRES GALVEZ,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,RUDY ADOLFO TOT OCH,HɃTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,RUDY ADOLFO TOT OCH,VICTOR HUGO DOM͎GUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9326,19-09-0040-46,19-021,ZACAPA,LA UNION,"LICEO PARTICULAR MIXTO ""JIREH""",BARRIO NUEVO,79418369,ASBEL IVAN SUCHITE ARROYO,ANA MAŔ CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ZACAPA
9327,19-09-0048-46,19-021,ZACAPA,LA UNION,"LICEO PARTICULAR MIXTO "" JIREH""",BARRIO NUEVO,79418369,ASBEL IVAN SUCHITE ARROYO,ANA MAŔ CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (UN D́ A LA SEMANA),ZACAPA
9328,19-10-0013-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO,BARRIO BUENOS AIRES,47097386,SILDY MARIELA PEREZ FRANCO,MARLON JOSUɠARCHILA LORENZO,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),ZACAPA
9329,19-10-1009-46,19-015,ZACAPA,HUITE,INSTITUTO MIXTO DE EDUCACION DIVERSIFICADA POR...,BARRIO EL CAMPO,55958103,SILDY MARIELA PEREZ FRANCO,ROBIDIO PORTILLO SALGUERO,DIVERSIFICADO,COOPERATIVA,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),ZACAPA


### Amount of columns

In [347]:
print("Number of columns:", len(education.columns))

print(education.columns)

Number of columns: 17
Index(['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO',
       'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR',
       'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL'],
      dtype='object')


In [348]:
education['DISTRITO'] = education['DISTRITO']

## Dataset Transformations

#### DISTRITO

Replace every null value with the first 3 characters of the district to identify in the department

In [349]:
# Function to replace NaN with first 3 chars of previous entry
def replace_missing_values(df, column_name):
    for i in range(1, len(df)):
        if pd.isna(df.loc[i, column_name]):
            if not pd.isna(df.loc[i-1, column_name]):
                df.loc[i, column_name] = df.loc[i-1, column_name][:3]
    return df

# Replace missing values
df = replace_missing_values(education, 'DISTRITO')

#### DEPARTAMENTO

Replace every space for '_' symbol and replace every lowercase value with uppercase value.

In [350]:
# Reemplazamos
education['DEPARTAMENTO'] = education['DEPARTAMENTO'].replace(' ', '_', regex=True)
education['DEPARTAMENTO'] = education['DEPARTAMENTO'].str.upper()

#### MUNICIPIO

Replace every space with '_' symbol.

In [351]:
education['MUNICIPIO'] = education['MUNICIPIO'].replace(' ', '_', regex=True)

#### ESTABLECIMIENTO: 

Replace unknown symbols and writing mistakes

In [352]:
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('я', 'ÑO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ӎ', 'ON', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ŕ', 'RIA', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ɖ', 'RE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ɓ', 'E', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('F́', 'FIA', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ǵ', 'GICO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ɒ', 'ER', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ɇ', 'EG', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ӈ', 'OG', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ƀ', 'EC', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ɘ', '', regex=True)#######
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('R͓', 'RIS', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('L͔', 'LIT', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('U͎', 'UIN', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('T͆', 'TIF', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ӌ', 'OL', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ڂ', 'UB', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ڍ', 'UM', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('R͑', 'RIQ', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ړ', 'US', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('D͎', 'DIN', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('T͎', 'TIN', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ӂ', 'OB', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ɏ', 'LO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('J̓', 'PAJ', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('A͓', 'AIS', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('/td>', '', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('Ɍ', 'EM', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('JOSɠ', 'JOSE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('JOSɢ', 'JOSE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('JOSɼ', 'JOSE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('TECPN', 'TECPAN', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ENSEсNZA', 'ENSEÑANZA', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('EDUCACIҎ', 'EDUCACION', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('BSICA', 'BASICA', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('TECNӌOGICO', 'TECNOLOGICO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('EVANGȌICO', 'EVANGELICO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('EVANGɌICO', 'EVANGELICO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('EVAANGELICO', 'EVANGELICO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('BILING܅', 'BILINGUE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('TRILING܅', 'TRILINGUE', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ASOCAICION', 'ASOCIACION', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('ASOCIACIɎ', 'ASOCIACION', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('INTITUTO', 'INSTITUTO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('INFORMTICA', 'INFORMATICA', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('GETSEMAN͢', 'GETSEMANI', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('BERNABɼ', 'BERNABO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('NGEL', 'ANGEL', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('SARAZځ', 'SARAZO', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('LVAREZ', 'ALVAREZ', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.replace('SINA͢', 'SINA', regex=True)

Delete quotes, single quotes and hyphen

In [353]:
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace("'", '', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace('-', '', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace('"', '', regex=True)

Delete parenthesis and commas

In [354]:
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace({r'[(),]': ''}, regex=True)

Delete doble spaces and change them to underscore

In [355]:
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace('  ', ' ', regex=True)
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].replace(' ', '_', regex=True)

Change everything to uppercase

In [356]:
education['ESTABLECIMIENTO'] = education['ESTABLECIMIENTO'].str.upper()

#### DIRECCIÓN: 

In [357]:
def replace_dot_with_nan(value):
    if isinstance(value, str) and (value == '.' or value == '-' or value == '--' or value == '---') and len(value) < 4:
        return np.nan
    return value

In [358]:
education['DIRECCION'] = education['DIRECCION'].apply(replace_dot_with_nan)
education['DIRECCION'] = education['DIRECCION'].replace(np.nan, 'No Especificado', regex=True)

In [359]:
education['DIRECCION'] = education['DIRECCION'].str.replace('я', 'ÑO', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ӎ', 'ON', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ŕ', 'RIA', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ɖ', 'RE', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ɓ', 'E', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('F́', 'FIA', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ǵ', 'GICO', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ɒ', 'ER', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ɇ', 'EG', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ӈ', 'OG', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ƀ', 'EC', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ɘ', '', regex=True)#######
education['DIRECCION'] = education['DIRECCION'].str.replace('R͓', 'RIS', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('L͔', 'LIT', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('U͎', 'UIN', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('T͆', 'TIF', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ӌ', 'OL', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ڂ', 'UB', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ڍ', 'UM', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('R͑', 'RIQ', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('ړ', 'US', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('D͎', 'DIN', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('T͎', 'TIN', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ӂ', 'OB', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ɏ', 'LO', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('J̓', 'PAJ', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('A͓', 'AIS', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('/td>', '', regex=True)
education['DIRECCION'] = education['DIRECCION'].str.replace('Ɍ', 'EM', regex=True)

education['DIRECCION'] = education['DIRECCION'].replace('AVENIDA', 'AV.', regex=True)
education['DIRECCION'] = education['DIRECCION'].replace('AVE.', 'AV.', regex=True)
education['DIRECCION'] = education['DIRECCION'].replace('1RA', '1ERA', regex=True)

In [360]:
# education['DIRECCION'] = education['DIRECCION'].replace("'", '', regex=True)
# education['DIRECCION'] = education['DIRECCION'].replace('-', '', regex=True)
education['DIRECCION'] = education['DIRECCION'].replace('"', '', regex=True)

In [361]:
education['DIRECCION'] = education['DIRECCION'].replace({r'[(),]': ''}, regex=True)

In [362]:
education['DIRECCION'] = education['DIRECCION'].replace('  ', ' ', regex=True)
education['DIRECCION'] = education['DIRECCION'].replace(' ', '_', regex=True)

In [363]:
education['DIRECCION'] = education['DIRECCION'].str.upper()

#### TELÉFONO

Delete all other symbol that is not a number from the row value

In [364]:
education['TELEFONO'] = education['TELEFONO'].replace({r'[.,E+/-]': ''}, regex=True)

Replace all null values for a row of 0s

In [365]:
education['TELEFONO'] = education['TELEFONO'].replace(np.nan, '00000000')

Fill the missing values of numbers that don't get to the standard 8 numbers

In [366]:
education['TELEFONO'] = education['TELEFONO'].apply(lambda x: x[:8].zfill(8) if isinstance(x, str) else x)

Shortend all values with more than 8 numbers

In [367]:
education['TELEFONO'] = education['TELEFONO'].apply(lambda x: x[:8] if isinstance(x, str) else x)